In [5]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16742592699998966772
 xla_global_id: -1]

## Imports

Im [rlHelloWorld](experiments/rlHelloWorld.ipynb) wurde ein anderes reinforcement learning framework genutzt, welches glaube ich eng mit tensorflow zusammenarbeitet. Die Libraries unten sind aber die originalen von Tensorflow aus dem professionellen Cart Pole Tutorial von Tensorflow [Tutorial](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb)

In [6]:
import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.environments import utils

In [7]:
import reverb

In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

## Create Custom Environnement

In [128]:
class RPS_Game(py_environment.PyEnvironment):
    
  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=[0], maximum=[2], name='observation')
    self._state = 0
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = np.random.randint(0, 3)
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Rock = 0, Paper = 1, Scissors = 2
    reward = 0
    if action == 0:
        if self._state == 1:
            reward = -1
        elif self._state == 2:
            reward = 1
    elif action == 1:
        if self._state == 0:
            reward = 1
        elif self._state == 2:
            reward = -1
    elif action == 2:
        if self._state == 0:
            reward = -1
        elif self._state == 1:
            reward = 1
    else:
        raise ValueError('`action` should be 0 or 1.')
    
    self._episode_ended = True
    return ts.termination(np.array([self._state], dtype=np.int32), reward)
    

    ''' if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0) '''

In [129]:
env_py = RPS_Game()
env = tf_py_environment.TFPyEnvironment(env_py)

### Testing with the environnement

In [130]:
env.action_spec(), env.observation_spec()

(BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(2, dtype=int32)),
 BoundedTensorSpec(shape=(1,), dtype=tf.int32, name='observation', minimum=array([0], dtype=int32), maximum=array([2], dtype=int32)))

In [131]:
env.time_step_spec()

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(1,), dtype=tf.int32, name='observation', minimum=array([0], dtype=int32), maximum=array([2], dtype=int32)),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [132]:
cartPoleEnv = suite_gym.load("CartPole-v0")

In [133]:
cartPoleEnv.action_spec(), cartPoleEnv.observation_spec()

(BoundedArraySpec(shape=(), dtype=dtype('int64'), name='action', minimum=0, maximum=1),
 BoundedArraySpec(shape=(4,), dtype=dtype('float32'), name='observation', minimum=[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], maximum=[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]))

In [134]:
cartPoleEnv.reset()

TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([ 0.00525199,  0.03895897, -0.02802346, -0.04529826], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})

In [135]:
env.reset()

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[2]], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})

In [136]:
tfCartPoleEnv = tf_py_environment.TFPyEnvironment(cartPoleEnv)

In [137]:
tfCartPoleEnv.reset()

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.03948171, 0.01425867, 0.00426048, 0.03866223]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})

In [138]:
env_py._state, type(env_py._state)

(2, int)

## RL AI

### TF Model

In [139]:
fc_layer_params = (100, 50) # Die Anzahl der Dense Units in einem Layer
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

### Agent creation
Dieser Agent wird nun über das Tensorflow eigene RL System erstellt und nicht über eine dritte Software

In [140]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Random Policy

In [141]:
eval_policy = agent.policy # main policy
collect_policy = agent.collect_policy # secondary policy to collect data

In [142]:
random_policy = random_tf_policy.RandomTFPolicy(env.time_step_spec(),
                                                env.action_spec())

In [143]:
time_step = env.reset()
random_policy.action(time_step)

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, state=(), info=())

In [144]:
def compute_avg_return(environment, policy, num_episodes=10):
    
    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
            total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

In [145]:
compute_avg_return(env, random_policy, 1)

0.0

## Replay Buffer (Data Contaier) and Data collection

In [146]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=100000,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmp3v9u49jh.
[reverb/cc/platform/tfrecord_checkpointer.cc:386] Loading latest checkpoint from /tmp/tmp3v9u49jh
[reverb/cc/platform/default/server.cc:71] Started replay server on port 17554


In [147]:
py_driver.PyDriver(
    env_py,
    py_tf_eager_policy.PyTFEagerPolicy(
        random_policy, use_tf_function=True),
        [rb_observer],
        max_steps=100
    ).run(env_py.reset())

(TimeStep(
 {'discount': array(0., dtype=float32),
  'observation': array([0], dtype=int32),
  'reward': array(0., dtype=float32),
  'step_type': array(2, dtype=int32)}),
 ())

In [148]:
iter(replay_buffer.as_dataset()).next()

(Trajectory(
 {'action': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 1], dtype=int32)>,
  'discount': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 1.], dtype=float32)>,
  'next_step_type': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 0], dtype=int32)>,
  'observation': <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[2],
        [2]], dtype=int32)>,
  'policy_info': (),
  'reward': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 0.], dtype=float32)>,
  'step_type': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 2], dtype=int32)>}),
 SampleInfo(key=<tf.Tensor: shape=(2,), dtype=uint64, numpy=array([68393806668783593, 68393806668783593], dtype=uint64)>, probability=<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.01010101, 0.01010101])>, table_size=<tf.Tensor: shape=(2,), dtype=int64, numpy=array([99, 99])>, priority=<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1., 1.])>))

In [149]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2).prefetch(3)

dataset

<PrefetchDataset element_spec=(Trajectory(
{'action': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None),
 'discount': TensorSpec(shape=(64, 2), dtype=tf.float32, name=None),
 'next_step_type': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None),
 'observation': TensorSpec(shape=(64, 2, 1), dtype=tf.int32, name=None),
 'policy_info': (),
 'reward': TensorSpec(shape=(64, 2), dtype=tf.float32, name=None),
 'step_type': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None)}), SampleInfo(key=TensorSpec(shape=(64, 2), dtype=tf.uint64, name=None), probability=TensorSpec(shape=(64, 2), dtype=tf.float64, name=None), table_size=TensorSpec(shape=(64, 2), dtype=tf.int64, name=None), priority=TensorSpec(shape=(64, 2), dtype=tf.float64, name=None)))>

In [150]:
iterator = iter(dataset)
print(iterator)

In [151]:
next(iterator)

[reverb/cc/client.cc:165] Sampler and server are owned by the same process (1143) so Table uniform_table is accessed directly without gRPC.


(Trajectory(
 {'action': <tf.Tensor: shape=(64, 2), dtype=int32, numpy=
 array([[0, 2],
        [0, 2],
        [0, 2],
        [1, 0],
        [0, 0],
        [1, 2],
        [0, 1],
        [1, 1],
        [2, 1],
        [1, 0],
        [0, 0],
        [2, 0],
        [1, 1],
        [1, 0],
        [1, 2],
        [1, 2],
        [0, 2],
        [2, 2],
        [1, 1],
        [2, 2],
        [1, 2],
        [2, 1],
        [2, 2],
        [1, 2],
        [2, 0],
        [1, 1],
        [0, 1],
        [2, 1],
        [1, 0],
        [2, 0],
        [1, 2],
        [0, 2],
        [1, 0],
        [1, 0],
        [2, 0],
        [1, 0],
        [0, 0],
        [2, 0],
        [0, 0],
        [2, 2],
        [1, 2],
        [2, 2],
        [2, 0],
        [1, 1],
        [2, 0],
        [1, 2],
        [1, 2],
        [0, 1],
        [1, 1],
        [2, 1],
        [1, 0],
        [2, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [0, 0],
        [0, 0],
        [0, 0],


## AI Training

In [152]:
# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

In [153]:
# Reset the train step.
agent.train_step_counter.assign(0)

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=0>

In [154]:
# Evaluate the agent's policy once before training.
compute_avg_return(env, agent.policy, 10), agent.policy

(0.2, <tf_agents.policies.greedy_policy.GreedyPolicy at 0x7f7a280a9e20>)

In [155]:
avg_return = compute_avg_return(env, agent.policy, 100)
returns = [avg_return]

<font color='red'>Achtung: Bei den nächsten beiden Zellen muss das <bold>Python Environnement</bold> verwendet werden und nicht das Tensorflow Environnement</font>

In [156]:
# Reset the environment.
time_step = env_py.reset()

In [157]:
# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env_py,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=1)

In [158]:
for _ in range(200):
    
  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % 10 == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % 10 == 0:
    avg_return = compute_avg_return(env, agent.policy, 100)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

step = 10: loss = 0.4393332600593567
step = 10: Average Return = -0.05000000074505806
step = 20: loss = 0.48160040378570557
step = 20: Average Return = 0.3799999952316284
step = 30: loss = 0.5213533639907837
step = 30: Average Return = 0.5
step = 40: loss = 0.4458102881908417
step = 40: Average Return = 0.6600000262260437
step = 50: loss = 0.5037575364112854
step = 50: Average Return = 0.4000000059604645
step = 60: loss = 0.4755828380584717
step = 60: Average Return = 0.6899999976158142
step = 70: loss = 0.4983588755130768
step = 70: Average Return = 0.7200000286102295
step = 80: loss = 0.45242875814437866
step = 80: Average Return = 0.6200000047683716
step = 90: loss = 0.3597545027732849
step = 90: Average Return = 0.41999998688697815
step = 100: loss = 0.4036809802055359
step = 100: Average Return = 0.5699999928474426
step = 110: loss = 0.26815348863601685
step = 110: Average Return = 1.0
step = 120: loss = 0.47258347272872925
step = 120: Average Return = 1.0
step = 130: loss = 0.303

## Running the AI -> As an interface

In [175]:
class RPS_Game_Interface(RPS_Game):
    def _reset(self):
        self._state = int(input("Enter your choice: "))
        self._episode_ended = False
        return ts.restart(np.array([self._state], dtype=np.int32))

In [176]:
env_py = RPS_Game_Interface()
env = tf_py_environment.TFPyEnvironment(env_py)

In [178]:
time_step = env.reset()
print("Observation: ", time_step.observation)
action_step = agent.policy.action(time_step)
time_step = env.step(action_step.action)
print("Selected Action: ", action_step.action)
print("Reward Gained:", time_step.reward)

Observation:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)
Selected Action:  tf.Tensor([1], shape=(1,), dtype=int32)
Reward Gained: tf.Tensor([1.], shape=(1,), dtype=float32)


### Environnement debugging

In [103]:
env.reset()

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[0]], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})

In [180]:
state = int(input("State: "))
action = int(input("Action: "))
reward = 0
if action == 0:
    if state == 1:
        reward = -1
    elif state == 2:
        reward = 1
elif action == 1:
    if state == 0:
        reward = 1
    elif state == 2:
        reward = -1
elif action == 2:
    if state == 0:
        reward = -1
    elif state == 1:
        reward = 1
else:
    raise ValueError('`action` should be 0 or 1.')

reward

1